In [274]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [275]:

internal_urls = set()
external_urls = set()

In [276]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [277]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [278]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [279]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://louisville.edu/coronavirus"
internal_urls.add(base_url)

In [280]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='louisville.edu', path='/coronavirus', params='', query='', fragment='')
https://louisville.edu
louisville.edu/coronavirus
louisville.edu


In [281]:
# internal_urls

In [282]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: http://louisville.edu
[INT] Internal link: https://ulink.louisville.edu/
[INT] Internal link: https://hrp.louisville.edu/
[INT] Internal link: https://csprd.louisville.edu/
[INT] Internal link: https://financials.louisville.edu/
[INT] Internal link: https://blackboard.louisville.edu/
[INT] Internal link: https://louisville.edu/businessops/portal/
[INT] Internal link: https://louisville.edu/login
[INT] Internal link: http://louisville.edu/admissions/apply
[INT] Internal link: http://louisville.edu/admissions/apply/transfer/
[INT] Internal link: http://graduate.louisville.edu/
[INT] Internal link: http://louisville.edu/dental/dmd/admission/
[INT] Internal link: http://www.law.louisville.edu/admissions
[INT] Internal link: http://louisville.edu/medschool/apply
[INT] Internal link: http://louisville.edu/business/apply
[INT] Internal link: http://louisville.edu/admissions/apply/international
[INT] Internal link: http://louisville.edu/online/admissions
[INT] Internal lin

[INT] Internal link: https://business.louisville.edu/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/reinhardt-advising-center/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/uofl-reach-center/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/undergraduate-academic-guidelines/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/undergraduate-academic-guidelines/gpa-standings/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/undergraduate-academic-guidelines/applying-for-a-minor/
[INT] Internal link: https://business.louisville.edu/current-student-resources/advising-assistance/undergraduate-academic-guidelines/choosing-changing-majors/
[INT] 

[INT] Internal link: https://business.louisville.edu/response-to-kentucky-center-for-economic-policy-blog-on-taxing-betting-uniformly/
[INT] Internal link: https://business.louisville.edu/category/analytics-brief
[INT] Internal link: https://business.louisville.edu/category/equine-industry
[INT] Internal link: https://business.louisville.edu/analytics-brief-2019-02/
[INT] Internal link: https://business.louisville.edu/galbreath-laureate-b-wayne-hughes/
[INT] Internal link: https://business.louisville.edu/category/spring20-currency
[INT] Internal link: https://business.louisville.edu/back-in-the-saddle-again-equine-industry-program-alums-return-to-campus/
[INT] Internal link: https://business.louisville.edu/category/alumni
[INT] Internal link: https://business.louisville.edu/category/uncategorized
[INT] Internal link: https://business.louisville.edu/category/winter20
[INT] Internal link: https://business.louisville.edu/equine-industry-series-a-runaway-success/
[INT] Internal link: https

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://business.louisville.edu/academics-programs/graduate-programs/apply/
[INT] Internal link: https://business.louisville.edu/learnmore/fulltimemba/
[INT] Internal link: https://business.louisville.edu/learnmore/professionalmba/
[INT] Internal link: https://business.louisville.edu/learnmore/entrepreneurshipmba/
[INT] Internal link: https://business.louisville.edu/learnmore/globalmba/
[INT] Internal link: https://business.louisville.edu/learnmore/onlinemba/
[INT] Internal link: https://business.louisville.edu/learnmore/uoflmsba/
[INT] Internal link: https://business.louisville.edu/learnmore/mac/
[INT] Internal link: https://business.louisville.edu/academics-programs/graduate-programs/apply
[INT] Internal link: https://business.louisville.edu
[INT] Internal link: https://business.louisville.edu/academics-programs/graduate-programs/apply/tuition-and-costs/
[INT] Internal link: https://business.louisville.edu/academics-programs/graduate-programs/apply/academic-calen

[INT] Internal link: https://business.louisville.edu/academics-programs/entrepreneurship-programs/forcht-center/community-programs/rev-it/revit-expert/
[INT] Internal link: https://business.louisville.edu/academics-programs/entrepreneurship-programs/forcht-center/community-programs/rev-it/revit-entrepreneur-application/
[INT] Internal link: https://business.louisville.edu/academics-programs/entrepreneurship-programs/forcht-center/community-programs/ulaunch/ulaunch-application/
[INT] Internal link: https://business.louisville.edu/about/learning-at-cob/entrepreneurship/forcht-center-for-entrepreneurship/programs/small-business-institute/consult-mba/
[INT] Internal link: https://business.louisville.edu/academics-programs/entrepreneurship-programs/forcht-center/community-programs/launch-it/launchit-registration/
[+] Total External links: 84
[+] Total Internal links: 367
[+] Total: 451


In [ ]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,4):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        try:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
            if len(internal_urls)>1000:
                break
        except requests.exceptions.ConnectionError:
            continue

26 0
[INT] Internal link: https://louisville.edu/coronavirus/students/campus-life/campus/login
[INT] Internal link: https://louisville.edu/coronavirus/students/assets/campus/affiliated-properties
[INT] Internal link: http://louisville.edu/housing
[INT] Internal link: https://louisville.edu/housing
[INT] Internal link: https://louisville.edu/housing/resources/express-check-out
[INT] Internal link: https://louisville.edu/housing/resources/covid-19
[INT] Internal link: https://louisville.edu/housing/resources/covid-19-faq
[INT] Internal link: https://louisville.edu/coronavirus/financial
[INT] Internal link: https://louisville.edu/involvement/leadership/engage-lead-serve-board/cardinalcupboard/
[INT] Internal link: https://louisville.edu/coronavirus/students/community-resources
[INT] Internal link: http://louisville.edu/newcards/first-year/first-time-freshman-checklist/freshman-virtual-orientation
[INT] Internal link: https://louisville.edu/commencement
[INT] Internal link: https://library

[INT] Internal link: https://louisville.edu/coronavirus/employees/human-resources/
[+] NEW Total External links: 106
[+] NEW Total Internal links: 484
[+] NEW Total: 590
[+] NEW Total External links: 106
[+] NEW Total Internal links: 484
[+] NEW Total: 590
[INT] Internal link: https://louisville.edu/coronavirus/faq/
[INT] Internal link: https://louisville.edu/provost/travel/interim-travel-policy-for-covid-19
[INT] Internal link: https://louisville.edu/campushealth/information/coronavirus/students
[INT] Internal link: mailto://finaid@louisville.edu
[INT] Internal link: http://louisville.edu/hr/covid-19-hr/updated-telecommuting-guidelines-4.30
[INT] Internal link: https://louisville.edu/policies/policies-and-procedures/pageholder/pol-vehicle-use
[INT] Internal link: https://louisville.edu/delphi
[INT] Internal link: http://louisville.edu/hr/covid-19-hr/covid-19-hr-information
[INT] Internal link: http://louisville.edu/hr/covid-19-hr/epsla-fmla-expansion-employee-guidelines
[INT] Internal

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 112
[+] NEW Total Internal links: 516
[+] NEW Total: 628
[INT] Internal link: https://louisville.edu/coronavirus/students/campus-life/login
[+] NEW Total External links: 112
[+] NEW Total Internal links: 517
[+] NEW Total: 629
[+] NEW Total External links: 112
[+] NEW Total Internal links: 517
[+] NEW Total: 629
[+] NEW Total External links: 112
[+] NEW Total Internal links: 517
[+] NEW Total: 629
[INT] Internal link: https://louisville.edu/coronavirus/assets/Temporary_Telecommuting_Guidelines_for_Employees.pdf
[+] NEW Total External links: 114
[+] NEW Total Internal links: 518
[+] NEW Total: 632
[INT] Internal link: https://louisville.edu/coronavirus/employees/remote-work/remote-work/login
[+] NEW Total External links: 114
[+] NEW Total Internal links: 519
[+] NEW Total: 633
[+] NEW Total External links: 114
[+] NEW Total Internal links: 519
[+] NEW Total: 633
[+] NEW Total External links: 114
[+] NEW Total Internal links: 519
[+] NEW Total: 633
[INT] Int

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026
[+] NEW Total External links: 251
[+] NEW Total Internal links: 775
[+] NEW Total: 1026


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [ ]:
semifinal_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    
    
    semifinal_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)


    

print(semifinal_immediate_urls)
total_count = len(semifinal_immediate_urls)
print(total_count)

In [ ]:
final_immediate_urls = set()
for semifinal_immediate_url in semifinal_immediate_urls:
    if "tel:" not in semifinal_immediate_url:
        final_immediate_urls.add(semifinal_immediate_url)

print(final_immediate_urls)
total_count = len(final_immediate_urls)
print(total_count)       

In [ ]:

parent_folder = "Doctoral" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
    os.makedirs(f'{parent_folder}\\{name_of_folder}')

In [ ]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    if ' ' in i:
        continue
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
        print(f"{code} can_save - {i}")
    except (urllib.error.HTTPError,urllib.error.URLError) as e:
        print("discarded")
        webpage_cannot_save = webpage_cannot_save+1
        continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        

In [ ]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    if webpage[-4:] == ".pdf":
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
            file_name = "Home"
        pdf_urls.add(webpage)
        print(webpage)
        r = requests.get(url, stream=True)
        with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
            for chunk in r.iter_content(2000):
                fd.write(chunk)
        pdf_count = pdf_count + 1

print(f"Pdfs saved: {pdf_count}")

In [ ]:
final_immediate_urls = final_immediate_urls - pdf_urls
webpage_save = webpage_save - pdf_urls
web_save_count = len(webpage_save)
total_count = len(final_immediate_urls)
print(total_count) 

In [ ]:

#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

In [ ]:
final_webpage_save_count = webpage_save_count - pdf_count
print(final_webpage_save_count)

In [ ]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    try:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
    except:
        print('caught')

print("Saving Completed")

In [ ]:
count = 0
for i in final_immediate_urls:
    url = i
    file_name = url.replace(base_url,'')
    file_name = file_name.replace('/','')
    if file_name == '':
            file_name = "Home"
    count = count + 1
    print(f"{count} of {total_count}")
    print(f'Visiting {base_url_text}/{file_name}')
    print(f'...Taking a screenshot')
    driver.get(url)
    S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
    driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
    print(f'Screenshot of {file_name} page taken! \n')
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')